In [119]:
import tensorflow as tf
import numpy as np
import pandas as pd 
import os
import glob
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from collections import Counter

In [12]:
%run -i calculate_bpm.py

<Figure size 432x288 with 0 Axes>

In [13]:
bpm = calculate_bpm('finger-tip-2.mp4')

Extracting frames...
Done: extracted 631 frames; Took 19.61 seconds
Getting images...
Done
Finished Calculating BPM


In [120]:
model = tf.keras.models.load_model('ml_model')
model.summary()

Model: "sequential_12"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_48 (Dense)             (None, 8)                 32        
_________________________________________________________________
dense_49 (Dense)             (None, 10)                90        
_________________________________________________________________
dense_50 (Dense)             (None, 10)                110       
_________________________________________________________________
dense_51 (Dense)             (None, 4)                 44        
Total params: 276
Trainable params: 276
Non-trainable params: 0
_________________________________________________________________


In [148]:
def get_input_data(accel_path, barometer_path, gyro_path):
    files = [accel_path, barometer_path, gyro_path]
    data = []
    for f in files:
        csv = pd.read_csv(f, header = None)
        for item in csv.values[50:-50]:
            data.append(item)
            
    data = np.array(data)
    labels = pd.read_csv('data/new_sensors/labels.txt', header = None).values

    return data, labels

In [149]:
accel_path = 'data/new_sensors/23,04,22,07,23,44,Fast Walking,accelerometer.txt'
barometer_path = 'data/new_sensors/23,04,22,07,23,44,Fast Walking,barometer.txt'
gyro_path = 'data/new_sensors/23,04,22,07,23,44,Fast Walking,gyroscope.txt'

data, labels = get_input_data(accel_path, barometer_path, gyro_path)

In [150]:
data[0]

array([ 0.43540573, -0.85404205,  9.967018  ])

In [151]:
def make_prediction(data, labels):
    print('Activity recognition started...')
    # import model and make predictions on whole data
    probability_model = tf.keras.Sequential([model, tf.keras.layers.Softmax()])
    new_predictions = probability_model.predict(data)
    
    # get labels from one hot encoding
    ohe = OneHotEncoder(sparse=False)
    y_ohe = ohe.fit_transform(labels.reshape(-1, 1))

    # inverse the OHE back to labels and make a list from it
    inv = ohe.inverse_transform(y_ohe).ravel()
    activities = np.unique(inv)

    # 
    prediction_indices = []
    for i in range(len(new_predictions)):
        prediction_indices.append(np.argmax(new_predictions[i]))

    counter = 0
    max_num = prediction_indices[0]
    for item in prediction_indices:
        current_frequency = prediction_indices.count(i)
        if current_frequency > counter:
            counter = current_frequency
            max_num = item
 
    prediction_out = activities[max_num]
    print(prediction_out)
    print(prediction_indices)

In [153]:
labels

array([['Resting'],
       ['Resting'],
       ['Fast Walking'],
       ['Fast Walking'],
       ['Jogging'],
       ['Jogging'],
       ['Stair Climber'],
       ['Stair Climber']], dtype=object)

In [152]:
pred = make_prediction(data, labels)

Activity recognition started...
Resting
[2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,

In [39]:
labels

array([['Resting'],
       ['Resting'],
       ['Fast Walking'],
       ['Fast Walking'],
       ['Push ups'],
       ['Push ups'],
       ['Jogging'],
       ['Jogging'],
       ['Stair climber'],
       ['Stair climber']], dtype=object)

In [185]:
gender = 'man'
duration = 1
bpm = 65
weight = 55
age = 22

if gender == 'man':
    calories_burned = duration * (0.6309 * bpm + 0.1988 * weight + 0.2017 * age - 55.0969) / 4.184

calories_burned

0.30664435946462576